In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

query = '당신은 누구인가요?'

llm = ChatOpenAI(model="gpt-4o-mini")
llm.invoke(query)

In [ ]:
# State : 상태, 현재 Agent의 상태
# Node : Agent가 수행하는 작업, 함수
# Edge : Node와 Node를 연결하는 선, 함수의 연결 (실선)
# Conditional Edge : 조건부 Edge, 함수의 조건부 연결 (점선)

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage


class AgentState(TypedDict):
    messages: list[Annotated[AnyMessage, add_messages]]

In [ ]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(AgentState)

In [ ]:
def generate(state: AgentState) -> AgentState:
    messages = state['messages']
    ai_message = llm.invoke(messages)
    return {"messages": [ai_message]}

In [ ]:
from langgraph.graph import START, END

graph_builder.add_node('generate', generate)
graph_builder.add_edge(START, 'generate')
graph_builder.add_edge('generate', END)


In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

initial_state = {"messages": [HumanMessage(query)]}
graph.invoke(initial_state)